## This notebook checks how long offers stay on market

In [7]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from aws_utils import read_csv_as_dataframe
import pandas as pd
import s3fs
pd.set_option('display.max_colwidth', -1)

In [8]:
website="gumtree"
location="warszawa"
#Metadata
BUCKET = f's3://data-apartments/production/{website}/{location}/'
df_gumtree = read_csv_as_dataframe(bucket_path=BUCKET, filename=f'{website}_db.csv', timestamp_column_name='downloaded')
df_gumtree['website'] = 'gumtree'
df_gumtree.query('downloaded > "2019-05-23"', inplace=True)

d:\desktop\dev\scraper_otodom\otodom\.venv\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
df_gumtree.groupby(by=['item_id'])['downloaded'].count().sort_values(ascending=False).value_counts()

1    45228
2    576  
3    4    
5    1    
4    1    
Name: downloaded, dtype: int64

In [10]:
def create_similarity_metric(x):
    return x.short_description + x.district + str(x.area) + str(x.price)

df_gumtree['score'] = df_gumtree.apply(create_similarity_metric, axis=1)

In [11]:
df_gumtree.head()

,downloaded,url,short_description,district,area,rooms,price_per_m,price,owner,item_id,...,basement,garage,garden,balcony,terrace,auction,latitude,longitude,website,score
19195,2019-05-24,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/bialoleka-+-mieszkanie-2+pokojowe-54-m-parter-+-okazjonalna-cena/1005055542230912407250109,Białołęka - Mieszkanie 2-pokojowe · 54 m² · parter - okazjonalna cena,Białołęka,54.0,2 pokoje,6851.851852,370000,Agencja,1005055542230912407250109,...,False,True,False,True,False,False,52.328988,21.007679,gumtree,Białołęka - Mieszkanie 2-pokojowe · 54 m² · parter - okazjonalna cenaBiałołęka54.0370000
19196,2019-05-24,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/ochota/mieszkanie-warszawa-ochota-240m2-nr-brz+ms+100322+508/1005055351430911543980109,Mieszkanie Warszawa Ochota 240m2 (nr: BRZ-MS-100322-508),Ochota,240.0,5 pokoi,12833.333333,3080000,Agencja,1005055351430911543980109,...,False,False,False,False,True,False,52.210336,20.971221,gumtree,Mieszkanie Warszawa Ochota 240m2 (nr: BRZ-MS-100322-508)Ochota240.03080000
19197,2019-05-24,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/mieszkanie-warszawa-srodmiescie-50m2-nr-brz+ms+152654+16/1005055351370911543980109,Mieszkanie Warszawa Śródmieście 50m2 (nr: BRZ-MS-152654-16),Śródmieście,50.0,2 pokoje,15360.000000,768000,Agencja,1005055351370911543980109,...,False,False,False,False,False,False,52.229676,21.012229,gumtree,Mieszkanie Warszawa Śródmieście 50m2 (nr: BRZ-MS-152654-16)Śródmieście50.0768000
19198,2019-05-24,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wlochy/wlochy-klosia-kawalerka-36-4-m2-2010-rok/1005055348450911543980109,"Włochy, Kłosia, kawalerka, 36, 4 m2, 2010 rok",Włochy,37.0,Kawalerka lub garsoniera,10783.783784,399000,Agencja,1005055348450911543980109,...,False,False,False,False,False,False,52.179906,20.947816,gumtree,"Włochy, Kłosia, kawalerka, 36, 4 m2, 2010 rokWłochy37.0399000"
19199,2019-05-24,https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/bialoleka-flisakow-41-m2-2-pokoje-2019-r/1005055348350911543980109,"Białołęka, Flisaków, 41 m2, 2 pokoje, 2019 r.",Białołęka,41.0,2 pokoje,7768.292683,318500,Agencja,1005055348350911543980109,...,False,False,False,True,False,False,52.328988,21.007679,gumtree,"Białołęka, Flisaków, 41 m2, 2 pokoje, 2019 r.Białołęka41.0318500"


In [12]:
df_gumtree.groupby(by=['score']).count().sort_values(by=['item_id'], ascending=False)['downloaded'].value_counts()

1     11777
2     1737 
3     857  
4     633  
5     415  
6     297  
7     185  
8     137  
9     106  
10    97   
11    76   
19    75   
20    72   
21    72   
18    60   
23    59   
12    54   
22    54   
17    53   
14    50   
13    45   
24    44   
15    43   
16    38   
27    29   
25    29   
26    27   
28    20   
31    7    
29    6    
33    5    
35    4    
34    3    
30    3    
32    2    
36    1    
53    1    
45    1    
38    1    
Name: downloaded, dtype: int64

In [ ]:
df_gumtree.query('score == "Do kupienia nowoczesny, gotowy do zamieszkania dom w cichej i spokojnej okolicy Wilanowa.Wilanów235.02590000"')